In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from tqdm import tqdm
from itertools import product

In [2]:
NUM_USERS = 53897
NUM_ITEMSETS = 27694

In [3]:
train_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_training.csv', names=['user_id', 'itemset_id'])
valid_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_valid_query.csv', names=['user_id', 'itemset_id'])
valid_ans = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_valid_answer.csv', names=['answer'])
test_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_test_query.csv', names=['user_id', 'itemset_id'])

In [4]:
train_users = list(set(train_df['user_id'].tolist()))
train_dict = dict()
for train_user in tqdm(train_users):
    items = train_df[train_df['user_id']==train_user]['itemset_id'].tolist()
    train_dict.update({train_user:items})

train_user_itemset = np.zeros((NUM_USERS, NUM_ITEMSETS))
for i in range(len(train_user_itemset)):
    train_user_itemset[i,train_dict[i]] = 1
train_itemset_user = train_user_itemset.T

100%|██████████| 53897/53897 [00:35<00:00, 1509.25it/s]


In [5]:
mul_train_itemset_user = train_itemset_user@train_itemset_user.T
norm = np.sqrt(np.diag(mul_train_itemset_user))[:,None]@(np.sqrt(np.diag(mul_train_itemset_user))[:,None]).T
train_itemset_sim = np.divide(mul_train_itemset_user, norm)
train_itemset_sim = train_itemset_sim*(1-np.eye(train_itemset_sim.shape[0],train_itemset_sim.shape[0]))

In [6]:
results = []
for i in tqdm(range(len(valid_df['itemset_id'].tolist()))):

    res = np.dot(train_user_itemset[valid_df['user_id'][i]], train_itemset_sim[valid_df['itemset_id'][i]])
    results.append(res)

results = np.array(results)
results[results>0] = 1
a = results
b = np.array(valid_ans['answer'])

(a == b).sum()/len(a)

100%|██████████| 335940/335940 [02:02<00:00, 2733.50it/s]


0.60904328153837

In [7]:
mul_train_user_itemset = train_user_itemset@train_user_itemset.T
norm = np.sqrt(np.diag(mul_train_user_itemset))[:,None]@(np.sqrt(np.diag(mul_train_user_itemset))[:,None]).T
train_user_sim = np.divide(mul_train_user_itemset, norm)
train_user_sim = train_user_sim*(1-np.eye(train_user_sim.shape[0],train_user_sim.shape[0]))

In [8]:
results = []
for i in tqdm(range(len(valid_df['user_id'].tolist()))):

    res = np.dot(train_itemset_user[valid_df['itemset_id'][i]], train_user_sim[valid_df['user_id'][i]])
    results.append(res)

results = np.array(results)
results[results>0] = 1
a = results
b = np.array(valid_ans['answer'])

(a == b).sum()/len(a)

100%|██████████| 335940/335940 [01:52<00:00, 2991.39it/s] 


0.60904328153837